## Binary Classification Sonar Project 1 for the Navy:  Mines vs. Rocks




In [1]:
# All necessary imports here
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


Using TensorFlow backend.


In [0]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)


###Step 1. Description of the Dataset
As briefly described in the project manual Sonar dataset consists of mainly 208 observations with 60 input variables and 1 output variable. This project revolves around a binary classification problem as our model has to predict whether or not the detected object is a rock 'R' or a mine 'M'.

In [0]:
#loading the dataset
path_to_dataset = "https://raw.githubusercontent.com/mahrukh98/incredible-AI/master/datasets/sonar.csv"
dataframe = pd.read_csv(path_to_dataset, header=None)
dataset = dataframe.values
# splitting into input (X) and output (Y) variables
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]

In [4]:
Y

array(['R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M

As the output variables need to be numerical so as to be predicted by the model, we're using the Label Encoder class from scikit-learn, then fit_transform method to learn the labels' mean and standard deviation afterwards applying those transformations on the training dataset for encoded labels.

In [0]:
#encoder object instantiation of LabelEncoder class
encoder = LabelEncoder()
#encoded_Y consists of the transformed(scaled) labels
encoded_Y = encoder.fit_transform(Y)

Finally, we've received the encoded labels where 1 represents rock and 0 represents mine.

In [6]:
encoded_Y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

### Step 2. Baseline Neural Network Model Performance
Creating the baseline model with single, densely connected hidden layer of 60 hidden units (sames as the input variables or features) and randomly initialized weights. The hidden layer is passed through 'relu' activation function that zeroes-out the negative values and only keeps the positive values of computation. Output layer is passed through 'sigmoid' activation function that squashes the output between 0 and 1 as a probabilistic distribution. Finally, 'binary_crossentropy' loss function which is particular for the binary classification problems , 'Adam' optimizer along with recording accuracy metrics are reserved for compilation.

In [0]:
# baseline model
def create_baseline():
  # create model, write code below
  model = Sequential([
    Dense(60, input_shape=(60,)),
    Activation('relu'),
    Dense(1),
    Activation('sigmoid'),
   ])
	
	# Compile model, write code below
  model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

  return model

Now, evaluating the model using stratified cross-validation in the scikit-learn framework for that purpose we're using KerasClassifier wrapper class with the model creation function, number of epochs and batch_size as argument. 

In [8]:
# evaluate model with standardized dataset
#estimator object instantiation of KerasClassifier wrapper class
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, encoded_Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Results: 83.71% (6.13%)


### Step 3. Re-Run The Baseline Model With Data Preparation
Now, for data preparation we're using the standardization technique from StandardScaler class.
We're asked to train the standardization procedure on the training data within the pass of a cross-validation run and to use the trained standardization to prepare the “unseen” test fold using pipeline wrapper!

In [9]:
# evaluate baseline model with standardized dataset
np.random.seed(seed)
estimators = []
#scaler object instantiation of StandardScaler class
scaler = StandardScaler()
estimators.append(('standardize', scaler))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Standardized: 85.59% (7.46%)


### Step 4. Tuning Layers and Number of Neurons in The Model


> **4.1. Evaluate a Smaller Network** Let's see, what reducing the hidden units to 30 adds to our model's capability!







In [10]:
# smaller model
def create_smaller():
	# Create model
	model = Sequential([
    Dense(30, input_shape=(60,)),
    Activation('relu'),
    Dense(1),
    Activation('sigmoid'),
   ])
	# Compile model
	model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

	return model
# standardizing the data with data preparation during the k-fold cross-validation run
estimators = []
scaler = StandardScaler()
estimators.append(('standardize', scaler))
estimators.append(('mlp', KerasClassifier(build_fn=create_smaller, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))



Smaller: 83.09% (6.41%)






> ### 4.2. Evaluate a Larger Network 
Let's try another hidden layer of 30 hidden units after the first one with 60 hidden units!








In [11]:
# larger model
def create_larger():
	# create model
	model = Sequential([
    Dense(60, input_shape=(60,)),
    Activation('relu'),
    Dense(30, input_shape=(60,)),
    Activation('relu'),
    Dense(1),
    Activation('sigmoid'),
   ])
  
	# Compile model
	model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

	return model
estimators = []
scaler = StandardScaler()
estimators.append(('standardize', scaler))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))





Larger: 85.09% (7.25%)


### Step 5: Really Scaling up: developing a model that overfits 
Now, to figure out the strength of our model that it lies exactly right at the border between the underfitting and overfitting, we'll have to cross that border i.e. overfit the model. This can be achieved by:


1.   Adding layers ----- 4 layers + 1 output layer
2.   Increasing hidden units --- 120----->60------>30------>15------>1
3.   Training for more epochs ---150



In [12]:
# overfitted model
def create_overfitted_model():
	# create model
	model = Sequential([
    Dense(120, input_shape=(60,)),
    Activation('relu'),
    Dense(60, input_shape=(60,)),
    Activation('relu'),
    Dense(30, input_shape=(60,)),
    Activation('relu'),
    Dense(15, input_shape=(60,)),
    Activation('relu'),
    Dense(1),
    Activation('sigmoid'),
   ])
  
	# Compile model
	model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

	return model
estimators = []
scaler = StandardScaler()
estimators.append(('standardize', scaler))
estimators.append(('mlp', KerasClassifier(build_fn=create_overfitted_model, epochs=150, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("New overfitted model: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


New overfitted model: 87.97% (7.54%)


### Step 6: Tuning the Model 
Changing the optimizer from 'Adam' to 'rmsprop' here, though we can tune many other hyper-parameters as well.

In [13]:
# improved model
def create_model():
	# create model
	model = Sequential([
    Dense(120, input_shape=(60,)),
    Activation('relu'),
    Dense(60, input_shape=(60,)),
    Activation('relu'),
    Dense(30, input_shape=(60,)),
    Activation('relu'),
    Dense(15, input_shape=(60,)),
    Activation('relu'),
    Dense(1),
    Activation('sigmoid'),
   ])
  
	# Compile model
	model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

	return model
estimators = []
scaler = StandardScaler()
estimators.append(('standardize', scaler))
estimators.append(('mlp', KerasClassifier(build_fn=create_model, epochs=120, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Improved model: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Improved model: 85.97% (8.04%)


### Step 7: Rewriting the code using the Keras Functional API

In [0]:
import keras
from keras import layers

In [15]:
# creating functional API for baseline model 
def create_baseline_fn():
  inputs = keras.Input(shape = (60,))
  x = layers.Dense(60, activation='relu')(inputs)
  output = layers.Dense(1, activation='sigmoid')(x)

  model = keras.Model(inputs, output)
  
  model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])
  
  return model

# evaluate baseline model with standardized dataset
np.random.seed(seed)
estimators = []
# scaler object instantiation of StandardScaler class
scaler = StandardScaler()
estimators.append(('standardize', scaler))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline_fn, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Baseline Standardized model accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Baseline Standardized model accuracy: 85.59% (7.46%)


In [16]:
# creating functional API for smaller model 
def create_smaller_fn():
  inputs = keras.Input(shape = (60,))
  x = layers.Dense(30, activation='relu')(inputs)
  output = layers.Dense(1, activation='sigmoid')(x)

  model = keras.Model(inputs, output)
  
  model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])
  
  return model

# evaluate smaller model with standardized dataset
np.random.seed(seed)
estimators = []
# scaler object instantiation of StandardScaler class
scaler = StandardScaler()
estimators.append(('standardize', scaler))
estimators.append(('mlp', KerasClassifier(build_fn=create_smaller_fn, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Smaller Standardized model accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Smaller Standardized model accuracy: 85.06% (7.61%)


In [17]:
# creating functional API for larger model 
def create_larger_fn():
  inputs = keras.Input(shape = (60,))
  x = layers.Dense(60, activation='relu')(inputs)
  x = layers.Dense(30, activation='relu')(x)
  output = layers.Dense(1, activation='sigmoid')(x)

  model = keras.Model(inputs, output)
  
  model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])
  
  return model

# evaluate larger model with standardized dataset
np.random.seed(seed)
estimators = []
# scaler object instantiation of StandardScaler class
scaler = StandardScaler()
estimators.append(('standardize', scaler))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger_fn, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Larger Standardized model accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Larger Standardized model accuracy: 84.61% (6.36%)


In [18]:
# creating functional API for overfitted model 
def create_overfitted_fn():
  inputs = keras.Input(shape = (60,))
  x = layers.Dense(120, activation='relu')(inputs)
  x = layers.Dense(60, activation='relu')(x)
  x = layers.Dense(30, activation='relu')(x)
  x = layers.Dense(15, activation='relu')(x)
  output = layers.Dense(1, activation='sigmoid')(x)

  model = keras.Model(inputs, output)
  
  model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])
  
  return model

# evaluate overfitted model with standardized dataset
np.random.seed(seed)
estimators = []
# scaler object instantiation of StandardScaler class
scaler = StandardScaler()
estimators.append(('standardize', scaler))
estimators.append(('mlp', KerasClassifier(build_fn= create_overfitted_fn, epochs=150, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Overfitted Standardized model accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))



Overfitted Standardized model accuracy: 87.45% (6.66%)


In [19]:
# creating functional API for tuned/improved model 
def create_improved_fn():
  inputs = keras.Input(shape = (60,))
  x = layers.Dense(120, activation='relu')(inputs)
  x = layers.Dense(60, activation='relu')(x)
  x = layers.Dense(30, activation='relu')(x)
  x = layers.Dense(15, activation='relu')(x)
  output = layers.Dense(1, activation='sigmoid')(x)

  model = keras.Model(inputs, output)
  
  model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
  
  return model

# evaluate improved model with standardized dataset
np.random.seed(seed)
estimators = []
# scaler object instantiation of StandardScaler class
scaler = StandardScaler()
estimators.append(('standardize', scaler))
estimators.append(('mlp', KerasClassifier(build_fn= create_improved_fn, epochs=120, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Improved Standardized model accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))



Improved Standardized model accuracy: 87.02% (7.70%)


### Step 8: Rewriting the code by doing Model Subclassing

In [0]:
import keras
from keras import layers

In [21]:
# Creating subclass for baseline model
class Baseline(keras.Model):
  def __init__(self):
    super(Baseline, self).__init__()
    self.dense1 = layers.Dense(60, activation='relu')
    self.dense2 = layers.Dense(1, activation='sigmoid')

  def call(self, inputs):
    x = self.dense1(inputs)
    return self.dense2(x)
  
# DISCLAIMER!!!
# This part is inspired from the functional API style :D 
# As, build_fn needs callable function or class instance, we're generating another method which will also accompany the input shape not specified in the class and 
# compilation step

def create_Baseline_subclass():
  inputs = keras.Input(shape = (60,))
  model = Baseline()
  output = model.call(inputs)
  
  model = keras.Model(inputs, output)
  model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])
  return model

# evaluate baseline model with standardized dataset
np.random.seed(seed)
estimators = []
# scaler object instantiation of StandardScaler class
scaler = StandardScaler()
estimators.append(('standardize', scaler))
estimators.append(('mlp', KerasClassifier(build_fn= create_Baseline_subclass, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Baseline Standardized model accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Baseline Standardized model accuracy: 85.59% (7.46%)


In [22]:
# Creating subclass for smaller model
class Smaller(keras.Model):
  def __init__(self):
    super(Smaller, self).__init__()
    self.dense1 = layers.Dense(30, activation='relu')
    self.dense2 = layers.Dense(1, activation='sigmoid')

  def call(self, inputs):
    x = self.dense1(inputs)
    return self.dense2(x)
  
# DISCLAIMER!!!
# This part is inspired from the functional API style :D 
# As, build_fn needs callable function or class instance, we're generating another method which will also accompany the input shape not specified in the class and 
# compilation step

def create_Smaller_subclass():
  inputs = keras.Input(shape = (60,))
  model = Smaller()
  output = model.call(inputs)
  
  model = keras.Model(inputs, output)
  model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])
  return model

# evaluate smaller model with standardized dataset
np.random.seed(seed)
estimators = []
# scaler object instantiation of StandardScaler class
scaler = StandardScaler()
estimators.append(('standardize', scaler))
estimators.append(('mlp', KerasClassifier(build_fn= create_Smaller_subclass, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Smaller Standardized model accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Smaller Standardized model accuracy: 85.06% (7.61%)


In [23]:
# Creating subclass for larger model
class Larger(keras.Model):
  def __init__(self):
    super(Larger, self).__init__()
    self.dense1 = layers.Dense(60, activation='relu')
    self.dense2 = layers.Dense(30, activation='relu')
    self.dense3 = layers.Dense(1, activation='sigmoid')

  def call(self, inputs):
    x = self.dense1(inputs)
    x = self.dense2(x)
    return self.dense3(x)
  
# DISCLAIMER!!!
# This part is inspired from the functional API style :D 
# As, build_fn needs callable function or class instance, we're generating another method which will also accompany the input shape not specified in the class and 
# compilation step

def create_Larger_subclass():
  inputs = keras.Input(shape = (60,))
  model = Larger()
  output = model.call(inputs)
  
  model = keras.Model(inputs, output)
  model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])
  return model

# evaluate larger model with standardized dataset
np.random.seed(seed)
estimators = []
# scaler object instantiation of StandardScaler class
scaler = StandardScaler()
estimators.append(('standardize', scaler))
estimators.append(('mlp', KerasClassifier(build_fn= create_Larger_subclass, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Larger Standardized model accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Larger Standardized model accuracy: 84.61% (6.36%)


In [24]:
# Creating subclass for overfitting model
class Overfitted(keras.Model):
  def __init__(self):
    super(Overfitted, self).__init__()
    self.dense1 = layers.Dense(120, activation='relu')
    self.dense2 = layers.Dense(60, activation='relu')
    self.dense3 = layers.Dense(30, activation='relu')
    self.dense4 = layers.Dense(15, activation='relu')
    self.dense5 = layers.Dense(1, activation='sigmoid')

  def call(self, inputs):
    x = self.dense1(inputs)
    x = self.dense2(x)
    x = self.dense3(x)
    x = self.dense4(x)
    return self.dense5(x)
  
# DISCLAIMER!!!
# This part is inspired from the functional API style :D 
# As, build_fn needs callable function or class instance, we're generating another method which will also accompany the input shape not specified in the class and 
# compilation step

def create_Overfitted_subclass():
  inputs = keras.Input(shape = (60,))
  model = Overfitted()
  output = model.call(inputs)
  
  model = keras.Model(inputs, output)
  model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])
  return model

# evaluate overfitted model with standardized dataset
np.random.seed(seed)
estimators = []
# scaler object instantiation of StandardScaler class
scaler = StandardScaler()
estimators.append(('standardize', scaler))
estimators.append(('mlp', KerasClassifier(build_fn= create_Overfitted_subclass, epochs=150, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Overfitting Standardized model accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Overfitting Standardized model accuracy: 87.45% (6.66%)


In [25]:
# Creating subclass for improved/tuned model
class Improved(keras.Model):
  def __init__(self):
    super(Improved, self).__init__()
    self.dense1 = layers.Dense(120, activation='relu')
    self.dense2 = layers.Dense(60, activation='relu')
    self.dense3 = layers.Dense(30, activation='relu')
    self.dense4 = layers.Dense(15, activation='relu')
    self.dense5 = layers.Dense(1, activation='sigmoid')

  def call(self, inputs):
    x = self.dense1(inputs)
    x = self.dense2(x)
    x = self.dense3(x)
    x = self.dense4(x)
    return self.dense5(x)
  
# DISCLAIMER!!!
# This part is inspired from the functional API style :D 
# As, build_fn needs callable function or class instance, we're generating another method which will also accompany the input shape not specified in the class and 
# compilation step

def create_Improved_subclass():
  inputs = keras.Input(shape = (60,))
  model = Improved()
  output = model.call(inputs)
  
  model = keras.Model(inputs, output)
  model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
  return model

# evaluate improved model with standardized dataset
np.random.seed(seed)
estimators = []
# scaler object instantiation of StandardScaler class
scaler = StandardScaler()
estimators.append(('standardize', scaler))
estimators.append(('mlp', KerasClassifier(build_fn= create_Improved_subclass, epochs=120, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Improved Standardized model accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Improved Standardized model accuracy: 86.99% (6.80%)


### Step 9: Rewriting the code without using scikit-learn
Here, we're asked for selecting the best model and implementing its code + the k-fold cross validation without scikit-learn !

In [0]:
# Converting string labels into one-hot encoded labels either 0 or 1
encoded_labels = []
for i in range(len(Y)):
  if Y[i] == 'R':
    encoded_labels.append(1)
  else:
    encoded_labels.append(0)
    

In [31]:
encoded_labels_arr = np.array(encoded_labels)
encoded_labels_arr

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [0]:
# Creating model
def create_final_model():
  final_model = Sequential([
    Dense(120, input_shape=(60,)),
    Activation('relu'),
    Dense(60, input_shape=(60,)),
    Activation('relu'),
    Dense(30, input_shape=(60,)),
    Activation('relu'),
    Dense(15, input_shape=(60,)),
    Activation('relu'),
    Dense(1),
    Activation('sigmoid'),
   ])
  
	# Compiling model
  final_model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
  return final_model


Applying K-fold cross validation here

In [35]:
np.random.seed(seed)
k = 10
num_val_samples = len(dataset) // k
np.random.shuffle(dataset)
all_scores = []
num_epochs = 100

for i in range(k):
  print('processing fold #', i)

  # Preparing the validation data and properly partitioning training data
  val_X = X[num_val_samples * i:num_val_samples * (i + 1)]
  train_X = np.append(X[:num_val_samples * i], X[num_val_samples * (i + 1):], axis=0) 
    
  val_Y = encoded_labels_arr[num_val_samples * i:num_val_samples * (i + 1)]
  train_Y = np.append(encoded_labels_arr[:num_val_samples * i] , encoded_labels_arr[num_val_samples * (i + 1):], axis=0)
  # Building the Keras model (already compiled)
  model = create_final_model() 
  all_scores = model.fit(train_X,train_Y,epochs=num_epochs,batch_size=5,verbose=0,validation_data = (val_X,val_Y))
  
  # Saving state dictionary of model    
  history_dict = all_scores.history
  val_score = np.average(history_dict['val_acc'])

  print("Final improved model's accuracy: %.2f%% " % (val_score*100))

processing fold # 0
Final improved model's accuracy: 35.15% 
processing fold # 1
Final improved model's accuracy: 60.15% 
processing fold # 2
Final improved model's accuracy: 55.05% 
processing fold # 3
Final improved model's accuracy: 76.80% 
processing fold # 4
Final improved model's accuracy: 53.15% 
processing fold # 5
Final improved model's accuracy: 25.75% 
processing fold # 6
Final improved model's accuracy: 49.70% 
processing fold # 7
Final improved model's accuracy: 64.45% 
processing fold # 8
Final improved model's accuracy: 47.25% 
processing fold # 9
Final improved model's accuracy: 98.60% 


In [36]:
val_score.mean()*100

98.60000003129244